# Giochiamo con tanti dati
1. Cosa succede con i dati mancanti (anche dopo aver trasformato)
2. Come riempire i dati mancanti

### Opzione 1: useremo fillna() per aggiungere i valori mancanti
### Opzione 2: useremo scikit-learn per riempire i dati

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
car_sales_missing = pd.read_csv("car-sales-extended-missing-data.csv")
car_sales_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [2]:

car_sales_missing.dropna(subset=["Price"], inplace=True)
X = car_sales_missing.drop("Price",axis=1)
y = car_sales_missing["Price"]


### Riempio di dati mancanti con la libreri di scikit-learn
1. importo la libreria
2. aggiungo missing ai valori testuali, e la media ai valori numerici

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Definisco come riempire i dati
cat_imputer = SimpleImputer(strategy="constant", fill_value="mancante")
door_imputer = SimpleImputer(strategy="constant", fill_value=4)
numeric_imputer = SimpleImputer(strategy="mean")

# Definisco le colonne
cat_features = ["Make","Colour"]
door_features = ["Doors"]
numeric_features = ["Odometer (KM)"]

# Creo un imputer (qualcosa che rimpe i dati mancanti)
imputer = ColumnTransformer([
    ("cat_imputer", cat_imputer, cat_features),
    ("door_imputer", door_imputer, door_features),
    ("numeric_imputer", numeric_imputer, numeric_features),
    
    ])

# Trasformo i dati
filled_X = imputer.fit_transform(X)
car_sales_filled = pd.DataFrame(filled_X, columns=["Make","Colour","Doors","Odometer (KM)"])
car_sales_filled.isna().sum()

Make             0
Colour           0
Doors            0
Odometer (KM)    0
dtype: int64

In [5]:
# Adesso possiamo avere i nostri dati come numeri e senza valori NULLI
categorical_feature = ["Make","Colour","Doors"]
one_hot = OneHotEncoder()
transformer = ColumnTransformer([("one_hot", one_hot, categorical_feature)],
                                remainder="passthrough")
transformed_X = transformer.fit_transform(car_sales_filled)
transformed_X

<950x15 sparse matrix of type '<class 'numpy.float64'>'
	with 3800 stored elements in Compressed Sparse Row format>

In [8]:
np.random.seed(42)
X_train,X_test,y_train,y_test = train_test_split(transformed_X,y, train_size=0.2)

In [16]:
model  =RandomForestRegressor(n_estimators=100)
model.fit(X_train,y_train)
model.score(X_test,y_test)


0.1967975931141922